# Seq2Seq + Attention 실습 노트북 (최종 버전)

이 노트북은 외부 데이터셋 없이 파이썬 리스트로 정의한 간단한 병렬 코퍼스를 사용해
Seq2Seq RNN + Attention 모델을 학습하고 번역 예시를 확인하는 예제입니다.

각 셀을 순서대로 실행하여 결과를 확인하세요.

모든 주요 단계에 상세한 주석을 추가하여 코드 이해도를 높였습니다.
Teacher Forcing 절충(tf_ratio=0.5) 및 hidden/cell 분리 맵핑 적용.

문제: 코드 중에서 어텐션 관련된 TODO 부분을 찾아 코드를 완성하고 전체 프로세스를 실행해 보시오.

In [2]:
### 1) 라이브러리 임포트 및 환경 설정
import random  # 시드 고정을 위한 random
import numpy as np  # 배열 연산
import torch  # PyTorch 기본 모듈
import torch.nn as nn  # 신경망 레이어 모듈
import torch.optim as optim  # 최적화 도구

# 재현성을 위한 시드 고정
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# GPU 사용 시 자동 선택, 없으면 CPU 사용
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")

Using device: cpu


In [3]:
### 2) 병렬 코퍼스 정의
# 영어 문장과 대응되는 프랑스어 문장 쌍을 리스트로 정의
pairs = [
    ('hello',       'bonjour'),    # 인사
    ('thank you',   'merci'),      # 감사 표현
    ('good night',  'bonne nuit'), # 작별 인사
    ('i am hungry', "j'ai faim"), # 상태 묘사
    ('i love you',  "je t'aime"),# 감정 표현
    ('how are you', 'comment ça va'), # 질문
]
print(f"총 문장 쌍 수: {len(pairs)}")

총 문장 쌍 수: 6


In [4]:
### 3) 토큰화 및 어휘집 구축
def tokenize(sentence):
    '''
    문자열을 소문자로 변경하고 공백으로 분할하여 토큰 리스트 반환
    e.g. 'Hello World' -> ['hello','world']
    '''
    return sentence.lower().split()

# 문장 시작/종료, 패딩 토큰 정의
SOS_TOKEN, EOS_TOKEN, PAD_TOKEN = '<sos>', '<eos>', '<pad>'

def build_vocab(sent_list):
    '''
    토큰 리스트에서 어휘집 생성
    special token: PAD=0, SOS=1, EOS=2
    '''
    vocab = {PAD_TOKEN: 0, SOS_TOKEN: 1, EOS_TOKEN: 2}
    for sent in sent_list:
        for tok in tokenize(sent):
            if tok not in vocab:
                vocab[tok] = len(vocab)
    return vocab

# 영어/프랑스어 텍스트 분리
src_texts = [src for src, _ in pairs]
trg_texts = [trg for _, trg in pairs]
# 어휘집 생성
SRC_VOCAB = build_vocab(src_texts)
TRG_VOCAB = build_vocab(trg_texts)
# 인덱스->토큰 매핑 생성
SRC_IVAL = {i:tok for tok,i in SRC_VOCAB.items()}
TRG_IVAL = {i:tok for tok,i in TRG_VOCAB.items()}

print(f"영어 vocab 크기: {len(SRC_VOCAB)} 토큰")
print(f"프랑스어 vocab 크기: {len(TRG_VOCAB)} 토큰")

영어 vocab 크기: 14 토큰
프랑스어 vocab 크기: 14 토큰


In [5]:
### 4) 인코딩 및 패딩
# 최대 시퀀스 길이 (SOS, EOS 포함)
MAX_SRC_LEN = max(len(tokenize(s)) + 2 for s in src_texts)
MAX_TRG_LEN = max(len(tokenize(s)) + 2 for s in trg_texts)

def encode_and_pad(sentence, vocab, max_len):
    '''
    문장 -> 인덱스 리스트 변환
    [SOS] + tokens + [EOS] + [PAD...]
    '''
    tokens = tokenize(sentence)
    idxs = [vocab[SOS_TOKEN]] + [vocab[t] for t in tokens] + [vocab[EOS_TOKEN]]
    # 부족한 길이만큼 PAD 추가
    idxs += [vocab[PAD_TOKEN]] * (max_len - len(idxs))
    return torch.LongTensor(idxs)

import torch
# 배치 데이터 생성 (모든 예시 사용)
dataset = [
    (encode_and_pad(s, SRC_VOCAB, MAX_SRC_LEN),
     encode_and_pad(t, TRG_VOCAB, MAX_TRG_LEN))
    for s, t in pairs
]
# 텐서로 변환 후 차원 Transpose: [batch, seq] -> [seq, batch]
src_batch = torch.stack([d[0] for d in dataset]).t().to(DEVICE)  # [src_len, batch]
trg_batch = torch.stack([d[1] for d in dataset]).t().to(DEVICE)  # [trg_len, batch]
print('src_batch shape:', src_batch.shape)
print('trg_batch shape:', trg_batch.shape)

src_batch shape: torch.Size([5, 6])
trg_batch shape: torch.Size([5, 6])


In [ ]:
### 5) 모델 정의
class Encoder(nn.Module):
    '''
    양방향 LSTM 인코더
    '''
    def __init__(self, input_dim, emb_dim, hid_dim):
        super().__init__()
        # 임베딩 레이어: 인덱스 -> 벡터
        self.embed = nn.Embedding(input_dim, emb_dim)
        # 2-way LSTM
        self.rnn   = nn.LSTM(emb_dim, hid_dim, bidirectional=True)
        # hidden, cell 분리 맵핑
        self.fc_h  = nn.Linear(hid_dim*2, hid_dim)
        self.fc_c  = nn.Linear(hid_dim*2, hid_dim)

    def forward(self, src):
        # src: [seq_len, batch]
        embedded = self.embed(src)  # [seq_len, batch, emb_dim]
        outputs, (h, c) = self.rnn(embedded)
        # 양방향 마지막 레이어 concat
        h_cat = torch.cat((h[-2], h[-1]), dim=1)  # [batch, hid_dim*2]
        c_cat = torch.cat((c[-2], c[-1]), dim=1)
        # 맵핑 후 unsqueeze
        h0 = torch.tanh(self.fc_h(h_cat)).unsqueeze(0)  # [1, batch, hid_dim]
        c0 = torch.tanh(self.fc_c(c_cat)).unsqueeze(0)
        return outputs, h0, c0

class Attention(nn.Module):
    '''
    Bahdanau Attention
    '''
    def __init__(self, hid_dim):
        super().__init__()
        self.attn = nn.Linear(hid_dim*3, hid_dim)
        self.v    = nn.Linear(hid_dim, 1, bias=False)

    def forward(self, hidden, enc_outs):
        # hidden: [1, batch, hid_dim]
        # enc_outs: [src_len, batch, hid_dim*2]
        src_len, batch, _ = enc_outs.shape
        # repeat hidden across src_len
        hidden_rep = hidden.repeat(src_len, 1, 1)  # [src_len, batch, hid_dim]
        # energy: alignment scores
        energy = torch.tanh(self.attn(torch.cat((hidden_rep, enc_outs), dim=2)))
        # compute attention weights
        return torch.softmax(self.v(energy).squeeze(2), dim=0)  # [src_len, batch]

class Decoder(nn.Module):
    '''
    어텐션 기반 디코더
    '''
    def __init__(self, output_dim, emb_dim, hid_dim, attention):
        super().__init__()
        self.embed = nn.Embedding(output_dim, emb_dim)
        self.rnn   = nn.LSTM(hid_dim*2 + emb_dim, hid_dim)
        self.attn  = attention
        self.out   = nn.Linear(hid_dim*3 + emb_dim, output_dim)

    def forward(self, input, hidden, cell, enc_outs):
        # input: [batch] 단일 timestep
        input = input.unsqueeze(0)  # [1, batch]
        emb   = self.embed(input)  # [1, batch, emb_dim]
        # 어텐션 가중치
        # TODO
        # reshape for bmm
        a     = a.unsqueeze(1).permute(2, 1, 0)  # [batch,1,src_len]
        enc   = enc_outs.permute(1, 0, 2)       # [batch,src_len,hid_dim*2]
        # context vector
        ctx   = torch.bmm(a, enc).permute(1, 0, 2)  # [1,batch,hid_dim*2]
        # concatenate emb and context
        # TODO
        # TODO
        output = output.squeeze(0)  # [batch, hid_dim]
        emb    = emb.squeeze(0)     # [batch, emb_dim]
        ctx    = ctx.squeeze(0)     # [batch, hid_dim*2]
        # 최종 예측
        pred   = self.out(torch.cat((output, ctx, emb), dim=1))  # [batch, output_dim]
        return pred, h, c

class Seq2Seq(nn.Module):
    '''
    Encoder-Decoder 통합
    '''
    def __init__(self, encoder, decoder, device, tf_ratio=0.5):
        super().__init__()
        self.encoder  = encoder
        self.decoder  = decoder
        self.device   = device
        self.tf_ratio = tf_ratio  # teacher forcing 비율

    def forward(self, src, trg):
        # src: [src_len, batch], trg: [trg_len, batch]
        trg_len, batch = trg.shape
        vocab_size    = self.decoder.out.out_features
        # 저장용 텐서 초기화
        outputs       = torch.zeros(trg_len, batch, vocab_size).to(self.device)
        # 인코더 실행
        enc_outs, h, c = self.encoder(src)
        # 첫 입력 토큰: <sos>
        input_tok      = trg[0]
        for t in range(1, trg_len):
            # 디코더 한 스텝 실행
            pred, h, c = self.decoder(input_tok, h, c, enc_outs)
            outputs[t] = pred
            # teacher forcing: 절반 확률로 정답 사용
            teacher = random.random() < self.tf_ratio
            input_tok = trg[t] if teacher else pred.argmax(1)
        return outputs

In [7]:
### 6) 모델 초기화 및 학습 설정
INPUT_DIM = len(SRC_VOCAB)
OUTPUT_DIM= len(TRG_VOCAB)
ENC_EMB_DIM, DEC_EMB_DIM, HID_DIM = 32, 32, 64

encoder  = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM)
attention= Attention(HID_DIM)
decoder  = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, attention)
model    = Seq2Seq(encoder, decoder, DEVICE, tf_ratio=0.5).to(DEVICE)

optimizer= optim.Adam(model.parameters(), lr=1e-3)
PAD_IDX  = TRG_VOCAB[PAD_TOKEN]
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

print(model)

Seq2Seq(
  (encoder): Encoder(
    (embed): Embedding(14, 32)
    (rnn): LSTM(32, 64, bidirectional=True)
    (fc_h): Linear(in_features=128, out_features=64, bias=True)
    (fc_c): Linear(in_features=128, out_features=64, bias=True)
  )
  (decoder): Decoder(
    (embed): Embedding(14, 32)
    (rnn): LSTM(160, 64)
    (attn): Attention(
      (attn): Linear(in_features=192, out_features=64, bias=True)
      (v): Linear(in_features=64, out_features=1, bias=False)
    )
    (out): Linear(in_features=224, out_features=14, bias=True)
  )
)


In [8]:
### 7) 학습 루프 (500 에폭)
model.train()
for epoch in range(1, 501):
    optimizer.zero_grad()
    outputs  = model(src_batch, trg_batch)
    # Loss 계산: 예측과 정답 reshape
    preds    = outputs[1:].reshape(-1, outputs.shape[-1])
    targets  = trg_batch[1:].reshape(-1)
    loss     = criterion(preds, targets)
    loss.backward()  # 역전파
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1)  # gradient clip
    optimizer.step()  # 파라미터 업데이트
    if epoch % 100 == 0:
        print(f"Epoch {epoch:03d} | Loss: {loss.item():.4f}")

Epoch 100 | Loss: 0.0206
Epoch 200 | Loss: 0.0050
Epoch 300 | Loss: 0.0024
Epoch 400 | Loss: 0.0015
Epoch 500 | Loss: 0.0010


In [9]:
### 8) 번역 예시 및 결과 확인
def translate(sentence):
    '''
    한 문장을 입력받아 번역 결과 반환
    '''
    model.eval()
    tokens = tokenize(sentence)
    # input tensor 생성
    idxs   = [SRC_VOCAB[SOS_TOKEN]] + [SRC_VOCAB.get(t, 0) for t in tokens] + [SRC_VOCAB[EOS_TOKEN]]
    tensor = torch.LongTensor(idxs).unsqueeze(1).to(DEVICE)
    # 인코더 실행
    enc_outs, h, c = encoder(tensor)
    trg_idxs = [TRG_VOCAB[SOS_TOKEN]]
    for _ in range(MAX_TRG_LEN):
        prev = torch.LongTensor([trg_idxs[-1]]).to(DEVICE)
        pred, h, c = decoder(prev, h, c, enc_outs)
        nxt = pred.argmax(1).item()
        trg_idxs.append(nxt)
        if nxt == TRG_VOCAB[EOS_TOKEN]:
            break
    # index -> token 변환 및 문자열 반환
    return ' '.join([TRG_IVAL[i] for i in trg_idxs[1:-1]])

for src, _ in pairs:
    print(f"{src:12s} → {translate(src)}")

hello        → bonjour
thank you    → merci
good night   → bonne nuit
i am hungry  → j'ai faim
i love you   → je t'aime
how are you  → comment ça va
